# azure_ai_agents_streaming.py - ELI5 Walkthrough
This notebook gives a super-friendly tour of `python/samples/getting_started/workflows/agents/azure_ai_agents_streaming.py` from the Agent Framework samples.


## Big Picture
Imagine two coworkers: a Writer who drafts marketing slogans and a Reviewer who reacts in real time. This workflow wires both Azure AI agents together and streams their thoughts token by token so you can watch the creative back-and-forth live.


## Key Ingredients
- **AzureAIAgentClient** - spins up hosted Azure AI agents with instructions.
- **AsyncExitStack** - keeps async context managers tidy and closes everything once we're done.
- **WorkflowBuilder.add_agent** - drops ready-to-use agent executors into the workflow graph.
- **run_stream() + AgentRunUpdateEvent** - streams incremental text chunks you can print as they arrive.
- **WorkflowOutputEvent** - emits the reviewer's final verdict as the workflow output.


### Workflow Diagram
```mermaid
flowchart LR
    Start(["Start Prompt"]) --> Writer[[Writer agent]]
    Writer --> Reviewer[[Reviewer agent]]
    Reviewer --> Output(["Workflow Output"])
```


In [ ]:
import asyncio
from collections.abc import Awaitable, Callable
from contextlib import AsyncExitStack
from typing import Any, Optional

from agent_framework import AgentRunUpdateEvent, WorkflowBuilder, WorkflowOutputEvent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from dotenv import load_dotenv

load_dotenv()

async def create_azure_ai_agent() -> tuple[Callable[..., Awaitable[Any]], Callable[[], Awaitable[None]]]:
    """Helper method to create an Azure AI agent factory and a close function."""
    stack = AsyncExitStack()
    cred = await stack.enter_async_context(AzureCliCredential())
    client = await stack.enter_async_context(AzureAIAgentClient(async_credential=cred))

    async def agent(**kwargs: Any) -> Any:
        return await stack.enter_async_context(client.create_agent(**kwargs))

    async def close() -> None:
        await stack.aclose()

    return agent, close


async def main() -> None:
    agent, close = await create_azure_ai_agent()
    try:
        writer = await agent(
            name="Writer",
            instructions=(
                "You are an excellent content writer. You create new content and edit contents based on the feedback."
            ),
        )
        reviewer = await agent(
            name="Reviewer",
            instructions=(
                "You are an excellent content reviewer. "
                "Provide actionable feedback to the writer about the provided content. "
                "Provide the feedback in the most concise manner possible."
            ),
        )
        workflow = (
            WorkflowBuilder()
            .add_agent(writer, id="Writer")
            .add_agent(reviewer, id="Reviewer", output_response=True)
            .set_start_executor(writer)
            .add_edge(writer, reviewer)
            .build()
        )

        last_executor_id: Optional[str] = None
        events = workflow.run_stream("Create a slogan for a new electric SUV that is affordable and fun to drive.")
        async for event in events:
            if isinstance(event, AgentRunUpdateEvent):
                eid = event.executor_id
                if eid != last_executor_id:
                    if last_executor_id is not None:
                        print()
                    print(f"{eid}:", end=" ", flush=True)
                    last_executor_id = eid
                print(event.data, end="", flush=True)
            elif isinstance(event, WorkflowOutputEvent):
                print("\n===== Final output =====")
                print(event.data)
    finally:
        await close()


# main() is left defined but not executed so you can control when to run the Azure workflow.


## What the script does
This sample links a writer and reviewer Azure AI agent with streaming updates.


## Setting up the agents safely
create_azure_ai_agent() hides the boilerplate for spinning up Azure CLI auth and the Azure AI agent client. It returns a factory that the workflow uses to create agents on demand, plus a close() helper so everything shuts down cleanly.

The async exit stack tracks every context manager so a single await close() unwinds credentials, clients, and agents even if an error appears mid-run.


In [ ]:
import inspect
print(inspect.getsource(create_azure_ai_agent))


## Building the workflow pipeline
The workflow graph is tiny on purpose:
1. Kick off with the Writer agent (set as the start executor).
2. Pipe whatever the Writer produces straight into the Reviewer using .add_edge(writer, reviewer).
3. Mark the Reviewer with output_response=True so its final AgentRunResponse becomes a workflow output.

Adding real agent objects via add_agent means the framework handles run vs. stream wiring with no extra glue code from you.


In [ ]:
workflow_outline = [
    ('start', 'Writer', 'prompt -> draft'),
    ('Writer', 'Reviewer', 'stream drafts token-by-token'),
    ('Reviewer', 'workflow_output', 'final critique'),
]
workflow_outline


## Streaming events in action
When you call workflow.run_stream(...), the engine yields two kinds of events that this sample prints:
- AgentRunUpdateEvent - incremental text chunks produced by the current agent.
- WorkflowOutputEvent - the final answer that the Reviewer yields once the run is done.

The loop keeps track of which executor is currently speaking so the console output stays readable.


In [ ]:
from agent_framework import AgentRunResponseUpdate

mock_stream = [
    AgentRunUpdateEvent('Writer', AgentRunResponseUpdate(text='Drafting: Electric thrills for everyone.')),
    AgentRunUpdateEvent('Writer', AgentRunResponseUpdate(text=' Embrace the spark.')),
    AgentRunUpdateEvent('Reviewer', AgentRunResponseUpdate(text='Suggestion: Mention affordability sooner.')),
    WorkflowOutputEvent('Final tweak: Electric fun you can afford.', source_executor_id='Reviewer'),
]

last_executor = None
for event in mock_stream:
    if isinstance(event, AgentRunUpdateEvent):
        if event.executor_id != last_executor:
            if last_executor is not None:
                print()
            print("{}:".format(event.executor_id), end=" ", flush=True)
            last_executor = event.executor_id
        print(event.data.text, end="", flush=True)
    elif isinstance(event, WorkflowOutputEvent):
        print("\n===== Final output =====")
        print(event.data)


## Takeaways
- AsyncExitStack makes async resource cleanup painless even with multiple agents.
- add_agent(..., output_response=True) is how you surface the final reviewer message as workflow output.
- Streaming events let you build live dashboards or logs without waiting for the run to finish.
- You can scale the pattern with more agents (editor, fact-checker, translator) by chaining more add_agent and add_edge calls.


In [ ]:
import asyncio

# Helper for notebooks vs. scripts
loop = asyncio.get_event_loop()
if loop.is_running():
    # Jupyter/VS Code notebooks already have an event loop, so await directly.
    await main()
else:
    asyncio.run(main())
